<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Computação em Nuvem II
Caderno de **Exercícios**<br> 
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>AWS Lambda;</li>
  <li>AWS Step Functions;</li>
  <li>AWS EventBridge.</li>
</ol>

---

# **Exercícios**

## 1\. AWS Lambda

Replique as atividades do item 1.3 em sua conta.

> **Extração dos dados da B3 (API)**

In [ ]:
# -- Imports:
import json
import requests
from datetime import datetime

# -- API URL:
URL = 'https://www2.cetip.com.br/ConsultarTaxaDi/ConsultarTaxaDICetip.aspx'

# -- Consumindo os dados da API:
try:
  resposta = requests.get(URL)
  resposta.raise_for_status()
except Exception as exc:
  raise(exc)
else:
    dados_b3_raw = json.loads(resposta.text)

print(dados_b3_raw)

{'taxa': '13,65', 'dataTaxa': '01/09/2022', 'indice': '36.339,24', 'dataIndice': '02/09/2022'}


> **Transformação**

In [ ]:
dados_b3 = dados_b3_raw.copy()

# -- Substituindo pontos e víruglas:
dados_b3['taxa'] = dados_b3['taxa'].replace(',','.')
dados_b3['indice'] = dados_b3['indice'].replace('.','').replace(',','.')

# Convertendo as os "valores" das "keys" 'dataTaxa' e 'dataIndice' para datetime:
dados_b3['dataTaxa'] = datetime.strptime(dados_b3['dataTaxa'], '%d/%m/%Y').strftime('%Y-%m-%d')
dados_b3['dataIndice'] = datetime.strptime(dados_b3['dataIndice'], '%d/%m/%Y').strftime('%Y-%m-%d')
dados_b3

{'taxa': '13.65',
 'dataTaxa': '2022-09-01',
 'indice': '36339.24',
 'dataIndice': '2022-09-02'}

> **Criando uma data de refêrencia e um arquivo csv**

In [ ]:
# --  Data de referências (hoje: data de criação do notebook):
dados_b3['dataRef'] = datetime.now().strftime('%Y-%m-%d')

# -- Arquivo csv para os dados transformados:
dados_b3_csv = ','.join([valor for valor in dados_b3.values()])

> **Printando dados consumidos, dados com `dataRef` e o csv**

In [ ]:
print(f'Dados consumidos da API da B3:\n {dados_b3_raw} \n')
print(f'Dados transformados:\n {dados_b3} \n')
print(f'Dados no formato csv:\n {dados_b3_csv}')

Dados consumidos da API da B3:
 {'taxa': '13,65', 'dataTaxa': '01/09/2022', 'indice': '36.339,24', 'dataIndice': '02/09/2022'} 

Dados transformados:
 {'taxa': '13.65', 'dataTaxa': '2022-09-01', 'indice': '36339.24', 'dataIndice': '2022-09-02', 'dataRef': '2022-09-02'} 

Dados no formato csv:
 13.65,2022-09-01,36339.24,2022-09-02,2022-09-02


**AWS Lambda para o Bucket Bronze**

In [ ]:
import json
import logging
from datetime import datetime

import boto3
import urllib3
from botocore.exceptions import ClientError


def lambda_handler(event, context) -> bool:

  # -- Setup:
  URL = 'https://www2.cetip.com.br/ConsultarTaxaDi/ConsultarTaxaDICetip.aspx'
  BUCKET_BRONZE = 'vinicius-dados-b3-bronze'

  cliente = boto3.client('s3')

  data = datetime.now().strftime('%Y-%m-%d')
  nome_arquivo_json = f'dados-b3-{data}.json'

  # -- Extract:
  try:
    http = urllib3.PoolManager()
    response = http.request(url=URL, method='get')
  except Exception as exc:
    raise exc
  else:
    dados_b3 = json.loads(response.data.decode())
    logging.info(msg=dados_b3)

  # -- Load:
  try:
      with open(f'/tmp/{nome_arquivo_json}', mode='w', encoding='utf8') as fp:
          json.dump(dados_b3, fp)
      cliente.upload_file(Filename = f'/tmp/{nome_arquivo_json}', 
                          Bucket = BUCKET_BRONZE, 
                          Key = nome_arquivo_json)
  except ClientError as exc:
      raise exc

  return json.dumps(dict(status=True))

**Print relacionado:** `Teste-lambda-bronze.png`

**AWS Lambda para o Bucket Silver**

In [ ]:
import json
from datetime import datetime

import boto3
from botocore.exceptions import ClientError


def lambda_handler(event, context) -> bool:

  # -- Setup:
  BUCKET_BRONZE = 'vinicius-dados-b3-bronze'
  BUCKET_SILVER = 'vinicius-dados-b3-silver'

  cliente = boto3.client('s3')

  data = datetime.now().strftime('%Y-%m-%d')
  nome_arquivo_csv = f'dados-b3-{data}.csv'
  nome_arquivo_json = f'dados-b3-{data}.json'

  # -- Extract:
  cliente.download_file(BUCKET_BRONZE, nome_arquivo_json, f'/tmp/{nome_arquivo_json}')

  with open(f"/tmp/{nome_arquivo_json}", mode='r', encoding='utf8') as fp:
      dados_b3 = json.load(fp)

  # -- Transform:
  dados_b3['taxa'] = dados_b3['taxa'].replace(',', '.')
  dados_b3['indice'] = dados_b3['indice'].replace('.', '').replace(',', '.')

  dados_b3['dataTaxa'] = datetime.strptime(dados_b3['dataTaxa'], '%d/%m/%Y').strftime('%Y-%m-%d')
  dados_b3['dataIndice'] = datetime.strptime(dados_b3['dataIndice'], '%d/%m/%Y').strftime('%Y-%m-%d')

  # -- Load:
  try:
      with open(f'/tmp/{nome_arquivo_csv}', mode='w', encoding='utf8') as fp:
          fp.write(','.join([v for v in dados_b3.values()]))
      cliente.upload_file(Filename=f'/tmp/{nome_arquivo_csv}', 
                          Bucket=BUCKET_SILVER, 
                          Key=f'data_referencia={data}/{nome_arquivo_csv}')
  except ClientError as exc:
      raise exc

  return json.dumps(dict(status=True))

**Print relacionado:** `Teste-lambda-silver.png`

**AWS Lambda para o criar a tabela no AWS Athena**

In [ ]:
import json
from datetime import datetime

import boto3
from botocore.exceptions import ClientError

def lambda_handler(event, context) -> bool:

  # -- Setup:
  BUCKET_SILVER = 'vinicius-dados-b3-silver'

  query = f"""
  CREATE EXTERNAL TABLE IF NOT EXISTS dados_b3 (
    taxa double, 
    data_taxa string, 
    indice double, 
    data_indice string
  )
  PARTITIONED BY (
    data_referencia string
  )
  ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
  WITH SERDEPROPERTIES ('separatorChar'=',') 
  LOCATION 's3://{BUCKET_SILVER}/'
  """

  cliente = boto3.client('athena')

  # -- Create (SQL table):
  try:
    cliente.start_query_execution(
      QueryString=query, 
      ResultConfiguration={'OutputLocation': 's3://vinicius-query-results/'}
    )
  except ClientError as exc:
    raise exc

  # -- Loading the partition:
  try:
    cliente.start_query_execution(
      QueryString='MSCK REPAIR TABLE dados_b3', 
      ResultConfiguration={'OutputLocation': 's3://vinicius-query-results/'}
    )
  except ClientError as exc:
    raise exc

  return json.dumps(dict(status=True))

**Print relacionados:** `Buckets-S3.png` e `AWS-Athena.png`.

## 2\. AWS Step Functions

Replique as atividades do item 2.3 em sua conta.

**Print relacionado**: `StepFunctions.png`.

## 3\. AWS EventBridge

Replique as atividades do item 3.3 em sua conta.

**Print relacionado:** `EventBridge.png`.